<a href="https://colab.research.google.com/github/Sreerag-Pillai/Data_Science/blob/main/Simple_moving_average_trading_bot_back_trader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas_ta backtrader ta numpy pandas matplotlib pandas_datareader scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218907 sha256=d7a2f41d7c7b2b550ddfb9ae0a8d3016a1506cb836b486f55ae479706c6856e6
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=336071cac4ed2489c930d8db127f3e17b85a2ac61884ec306ea40160499bba1b
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built pandas_ta ta


In [ ]:
#Importing necessary libraries
import backtrader as bt
import datetime
from datetime import time
import numpy as np
from scipy.signal import argrelextrema
import backtrader.indicators as btind
import backtrader as bt
from datetime import time
import pandas as pd
import math
import calendar
import pytz  # For timezone conversions

In [ ]:

class SMA_DollarVolume_Strategy(bt.Strategy):
    params = (
        ('sma1_period', 2),
        ('exit_time', datetime.time(14, 40)),
         ('market_open_time', datetime.time(9, 30)),  # Market open time
        ('market_close_time', datetime.time(16, 0)),
        ('five_point_stop', 0.03),
        ('take_profit', 0.1),
        ('same_day_exit_time', datetime.time(15, 00))
    )

    def __init__(self):
        self.sma1 = bt.indicators.SimpleMovingAverage(self.data.close, period=2)
        self.dollar_volume = ((self.data.open + self.data.high + self.data.low + self.data.close) / 4) * self.data.volume
        self.dol_vol_avg = bt.indicators.SMA(self.dollar_volume, period=65)
        self.order = None
        self.entry_price = None
        self.stop_loss_price = None
        self.take_profit_price = None
        self.position_size = 6
        self.order_pending = False
        self.trade_opened_today = False
        self.entry_datetime = None
        self.entry_executed = False
        self.trades_df = pd.DataFrame(
            columns=[
                'Trade', 'Entry Date', 'Entry Price', 'Exit Date', 'Exit Price', '% Change',
                'Profit', 'Profit %', 'Position Size', 'Position Value', 'Cumulative Profit'
            ]
        )
        self.cum_profit = 0.0

        self.thanksgiving_datetimes = {self.get_fourth_thursday_of_november(year) for year in range(2020, 2030)}  # Update the range as needed
        self.black_friday_datetimes = {self.get_black_friday(year) for year in range(2020, 2030)}  # Update the range as needed
        self.first_action = None

    @staticmethod
    def get_fourth_thursday_of_november(year):
        """Returns the datetime of the fourth Thursday of November for the given year."""
        # November is 11th month
        month = 11
        # Find out the first Thursday of November
        first_thursday = next(week[calendar.THURSDAY] for week in calendar.monthcalendar(year, month) if week[calendar.THURSDAY])
        # Fourth Thursday will be 21 days after the first Thursday
        fourth_thursday = first_thursday + 21
        # Returning as datetime object with time set to 00:00:00
        return datetime.datetime(year, month, fourth_thursday)

    @staticmethod
    def get_black_friday(year):
        """Returns the datetime of Black Friday for the given year."""
        # Black Friday is the day after Thanksgiving
        thanksgiving = SMA_DollarVolume_Strategy.get_fourth_thursday_of_november(year)
        black_friday = thanksgiving + datetime.timedelta(days=1)
        # Returning as datetime object with time set to 13:00:00 CST
        black_friday_cst = black_friday.replace(hour=13, minute=0, second=0)
        # Convert to CST timezone
        return pytz.timezone('America/Chicago').localize(black_friday_cst)

    def next(self):
        current_time = self.data.datetime.time(0)
        current_datetime = self.data.datetime.datetime(0)
        current_datetime_utc = self.data.datetime.datetime(0)
        current_datetime_cst = current_datetime_utc.astimezone(pytz.timezone('America/Chicago'))

         # Check if current time is within market hours
        if not (self.p.market_open_time <= current_time <= self.p.market_close_time):
            return

        # Check for Black Friday and stop trading at 1 PM CST
        if current_time >= self.p.same_day_exit_time:
            self.close_trade_if_open()
            self.trade_opened_today = False
            return



        # Entry conditions with prevention of overlapping trades
        if current_time < self.p.exit_time:
            if self.data.close[0] > self.sma1[0] * 0.98:
                if self.first_action != 'sell':  # Prevent overlapping buy orders
                    self.order = self.buy(size=self.position_size)
                    self.order_pending = True
                    if not self.first_action:
                        self.first_action = 'buy'
            elif self.data.close[0] < self.sma1[0] * 1.02:
                if self.first_action != 'buy':  # Prevent overlapping sell orders
                    self.order = self.sell(size=self.position_size)
                    self.order_pending = True
                    if not self.first_action:
                        self.first_action = 'sell'

          # Check for conditions to close the trade
        if self.position.size > 0 and self.entry_executed:  # If there is an open position
          if self.data.close[0] >= self.take_profit_price or self.data.close[0] <= self.stop_loss_price:
              self.close_trade_if_open()
        if self.position.size != 0 or self.order_pending:
            return


    def close_trade_if_open(self):
      if self.position.size != 0:
          exit_price = self.data.close[0]
          self.close()
          self.log_trade('exit', exit_price)
          self.entry_executed = False
          self.first_action = None   # Reset the flag after closing a position




    def log_trade(self, action, price):
      if action == 'entry':
          # Log the entry details
          self.entry_price = price
          self.entry_datetime = self.data.datetime.datetime(0)  # Store the entry date and time
          self.stop_loss_price = price - (price * self.p.five_point_stop)  # Calculate the stop loss price
          self.take_profit_price = price + (price * self.p.take_profit)   # Calculate the take profit price

          # Classify the trade type based on the first action
          trade_type = 'Long Entry' if self.first_action == 'buy' else 'Short Entry'

          # Create a dictionary to store the trade details
          trade_row = {
              'Trade': trade_type,
              'Entry Date': self.entry_datetime,
              'Entry Price': self.entry_price,
              'Exit Date': '',
              'Exit Price': '',
              '% Change': '',
              'Profit': '',
              'Profit %': '',
              'Position Size': abs(self.position_size),
              'Position Value': self.entry_price * abs(self.position_size),
              'Cumulative Profit': self.cum_profit
          }

      elif action == 'exit':
          # Log the exit details
          exit_price = price
          exit_datetime = self.data.datetime.datetime(0)  # Store the exit date and time
          trade_type = 'Exit Long' if self.first_action == 'buy' else 'Exit Short'  # Determine the trade type
          percent_change = ((exit_price - self.entry_price) / self.entry_price) * 100 if self.entry_price else 0
          profit = (exit_price - self.entry_price) * abs(self.position_size) if self.entry_price else 0
          self.cum_profit += profit
          position_value = self.entry_price * abs(self.position_size) if self.entry_price else 0
          profit_percent = (profit / position_value) * 100 if position_value != 0 else 0

          # Create a dictionary to store the trade details
          trade_row = {
              'Trade': trade_type,
              'Entry Date': self.entry_datetime,
              'Entry Price': self.entry_price,
              'Exit Date': exit_datetime,
              'Exit Price': exit_price,
              '% Change': percent_change,
              'Profit': profit,
              'Profit %': profit_percent,
              'Position Size': abs(self.position_size),
              'Position Value': position_value,
              'Cumulative Profit': self.cum_profit
          }

      # Use pandas.concat to add the new row to the DataFrame
      trade_row_df = pd.DataFrame([trade_row])
      self.trades_df = pd.concat([self.trades_df, trade_row_df], ignore_index=True)


    def stop(self):
        if self.position.size != 0:
            self.log_trade('exit', self.data.close[0])

        print(self.trades_df)
        # Export trades DataFrame to CSV
        self.trades_df.to_csv('trade_log.csv', index=False)
        print("Trade log saved to trade_log.csv")


    def notify_order(self, order):
      if order.status in [order.Completed]:
          if order.isbuy():
              # Log entry trade for buy order
              self.log_trade('entry', order.executed.price)
              self.entry_executed = True  # Flag set to True after a buy order
          elif order.issell():
              # Log entry trade for sell order if it's the first action or exit trade if not
              if not self.entry_executed:
                  self.log_trade('entry', order.executed.price)
                  self.entry_executed = True  # Flag set to True after a sell order
              else:
                  self.log_trade('exit', order.executed.price)

          self.order = None  # Reset the order attribute




In [ ]:
class CustomFuturesCommisionScheme(bt.CommInfoBase):
    ''' Custom Commission Scheme for Futures Trading with specific margin requirements '''
    params = (
        ('commission', 2.10),
        ('margin', 500),
        ('mult', 1),
        ('commtype', bt.CommInfoBase.COMM_FIXED),
    )

    def _getcommission(self, size, price, pseudoexec):
        ''' Calculates the commission for a particular trade '''
        return abs(size) * self.p.commission


# Create cerebro instance
cerebro = bt.Cerebro()

# Load test data into a data feed and add to cerebro
# Load the dataset
df = pd.read_csv('GDBCup - Week2.csv')

# Concatenate the Date and Time to form a DateTime column
df['DateTime'] = pd.to_datetime(df['Date'].astype(str) + ' ' + df['Time'])
df.set_index('DateTime', inplace=True)

data_feed = bt.feeds.PandasData(dataname=df,sessionstart=datetime.time(8,30),sessionend=datetime.time(14,40))
#data_feed.addfilter(session_filter)

cerebro.adddata(data_feed)

# Add the strategies
cerebro.addstrategy(SMA_DollarVolume_Strategy)


custom_comm = CustomFuturesCommisionScheme(margin=500)  # $500 per contract margin
cerebro.broker.addcommissioninfo(custom_comm)

# Add constraints and set initial cash
cerebro.broker.set_cash(249993.0)

cerebro.addsizer(bt.sizers.SizerFix, stake=6)
#cerebro.broker.setcommission(commission=2.10, commtype=bt.CommInfoBase.COMM_FIXED)

print('Starting Portfolio Value : %.2f' % cerebro.broker.getvalue())
cerebro.run()

print('Ending Portfolio Value : %.2f' % cerebro.broker.getvalue())


#cerebro.plot()

Starting Portfolio Value : 249993.00
           Trade          Entry Date  Entry Price            Exit Date  \
0     Long Entry 2023-11-23 09:35:00      4571.25                        
1     Long Entry 2023-11-23 09:40:00      4571.75                        
2     Long Entry 2023-11-23 09:45:00      4572.00                        
3     Long Entry 2023-11-23 09:50:00      4571.75                        
4     Long Entry 2023-11-23 09:55:00      4572.00                        
..           ...                 ...          ...                  ...   
144   Long Entry 2023-11-28 14:30:00      4558.00                        
145   Long Entry 2023-11-28 14:35:00      4558.00                        
146   Long Entry 2023-11-28 14:40:00      4558.00                        
147    Exit Long 2023-11-28 14:40:00      4558.00  2023-11-28 15:00:00   
148  Short Entry 2023-11-28 15:05:00      4563.75                        

    Exit Price  % Change Profit  Profit % Position Size  Position Value  \